In [14]:
import pandas as pd

In [47]:
df1= pd.read_csv(r"C:\Users\thinh.lv\google-maps-scraper\src\csv\thẩm mỹ.csv")

In [48]:
df2= pd.read_csv(r"C:\Users\thinh.lv\google-maps-scraper\src\csv\làm đẹp.csv")

In [49]:
df3= pd.read_csv(r"C:\Users\thinh.lv\google-maps-scraper\src\csv\spa.csv")

In [52]:
result_df = pd.concat([df1, df2, df3]).drop_duplicates()

In [53]:
result_df.shape

(470, 12)

In [54]:
import os

In [56]:
# Đường dẫn đến thư mục chứa các file CSV
folder_path = 'C:/Users/thinh.lv/google-maps-scraper/src/csv'

# Khởi tạo một danh sách để lưu trữ các DataFrame
dfs = []

# Lặp qua tất cả các file trong thư mục
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Đọc từng file CSV và thêm vào danh sách
        df = pd.read_csv(file_path)
        dfs.append(df)

# Thực hiện union
result_df = pd.concat(dfs).drop_duplicates()

In [57]:
fnb = pd.read_excel("C:/Users/thinh.lv/google-maps-scraper/src/df_fnb_ninh_kieu.xlsx")

In [61]:
fnb = fnb.drop(columns=['categories'])

In [62]:
fnb.head(5)

,Unnamed: 0,place_id,name,main_category,phone,address,address_ward,address_street,address_city,address_postal_code,address_state,address_country_code
0,0,ChIJ1zgaajWTpzERAwdFNM7oyxM,Sailing Club Phú Quốc,Nhà hàng,0931 031 035,"Next to Intercontinental, Lô B7 Khu phức, Dươn...","Dương Tơ, Thành phố Phú Quốc","Next to Intercontinental, Lô B7 Khu phức",Kiên Giang,920000.0,"Tp. Phú Quốc, Kiên Giang",VN
1,1,ChIJzcWlVKlioDER8x5DgJZRmzc,Karaoke Paradise 1,Karaoke,0292 3797 797,"18 Lê Lợi, Cái Khế, Ninh Kiều, Cần Thơ","Cái Khế, Ninh Kiều",18 Lê Lợi,Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN
2,2,ChIJd3KrYa0odTER_3IUi3FL3s8,Birdy,Quán bar,0938 818 156,"80 Phạm Viết Chánh, Phường 19, Bình Thạnh, Thà...","Phường 19, Bình Thạnh",80 Phạm Viết Chánh,Thành phố Hồ Chí Minh,NaN,"Bình Thạnh, Thành phố Hồ Chí Minh",VN
3,3,ChIJcx38zHmJoDERHUu5SybjbXk,QUÁN NGON CẦN THƠ HÒA TUỘC,Nhà hàng,1900 866808,"333 Đ. Nguyễn Văn Linh, Phường An Khánh, Ninh ...","Phường An Khánh, Ninh Kiều",333 Đ. Nguyễn Văn Linh,Cần Thơ,94000.0,"Ninh Kiều, Cần Thơ",VN
4,4,ChIJ4-uslaNioDERJcQkA493bD4,Ninh Kieu Sky Bar,Quán bar,0939 786 123,"02 Hai Ba Trung Stress District City, Tân An, ...","Tân An, Ninh Kiều",02 Hai Ba Trung Stress District City,Cần Thơ,NaN,"Ninh Kiều, Cần Thơ",VN


In [60]:
result_df.head(5)

,Unnamed: 0,place_id,name,main_category,phone,address,address_ward,address_street,address_city,address_postal_code,address_state,address_country_code
0,0,ChIJhw8kJOurNTER6PEWU5OmVqM,Shop Bé Na,Cửa hàng quần áo trẻ em,0888 480 896,"Shop Bé Na, 91B P. Thợ Nhuộm, Trần Hưng Đạo, H...","Trần Hưng Đạo, Hoàn Kiếm",91B P. Thợ Nhuộm,Hà Nội,100000.0,"Hoàn Kiếm, Hà Nội",VN
1,15,ChIJk6LQQpyrNTERPJXcubeuOBw,Mẹo Nuôi Bé,Siêu thị,0943 235 558,"Mẹo Nuôi Bé, Ngõ 105 P. Tôn Đức Thắng, Hàng Bộ...","Hàng Bột, Đống Đa",Ngõ 105 P. Tôn Đức Thắng,Hà Nội,100000.0,"Đống Đa, Hà Nội",VN
2,30,ChIJdytDmXSsNTERYvRCAVxsLzY,Siêu thị mẹ bầu và em bé KidsPlaza Xã Đàn,Cửa hàng đồ trẻ em,0796 297 588,"Siêu thị mẹ bầu và em bé KidsPlaza Xã Đàn, 374...","Nam Đồng, Đống Đa",374 P. Xã Đàn,Hà Nội,100000.0,"Đống Đa, Hà Nội",VN
3,45,ChIJTZIb_XutNTERtvT08MlkgUk,Shop Mẹ Và Bé Love Baby,Cửa hàng đồ trẻ em,0975 275 820,"Shop Mẹ Và Bé Love Baby, 189 P. Quan Nhân, Nhâ...","Nhân Chính, Thanh Xuân",189 P. Quan Nhân,Hà Nội,NaN,"Thanh Xuân, Hà Nội",VN
4,60,ChIJt1_l6JirNTERCBvtxyCkNLg,Hệ Thống Cửa Hàng Mẹ và Bé Bibo Mart - Thế Giớ...,Cửa hàng đồ trẻ em,NaN,Hệ Thống Cửa Hàng Mẹ và Bé Bibo Mart - Thế Giớ...,"Làng Quốc tế Thăng Long, Cầu Giấy","Tòa B3, P. Vĩnh Hồ",Hà Nội,100000.0,"Cầu Giấy, Hà Nội",VN


In [63]:
result_df_1 = pd.concat([fnb, result_df]).drop_duplicates()

In [74]:
result_df_1.to_excel("C:/Users/thinh.lv/google-maps-scraper/src/df_fnb_ninh_kieu.xlsx")

In [66]:
# Tách cột 'address_ward' thành hai cột mới 'city' và 'ward'
result_df_1[['ward', 'state']] = result_df_1['address_ward'].str.rsplit(', ', n=1, expand=True)


In [71]:
import re

In [72]:
def cleaning_street(street):
    if isinstance(street, str):
        st = re.sub("(Hẻm|Ngõ|Ngách) \d+\/*\d*\/*\d* ", "", street)
        st = re.sub("(Hẻm T\d+)|Hẻm \d+[A-Za-z] ", "", st)
        st = re.sub(r"(30\/4|30\s(th[aá]ng\s)?4|30\s(th[aá]ng\s)?tư)", "Đường 30 Tháng 4", st, flags=re.IGNORECASE)
        st = re.sub(r"(3\/2|3\s(th[aá]ng\s)?2|3\s(th[aá]ng\s)?hai)", "Đường 3 Tháng 2", st, flags=re.IGNORECASE)
        st = re.sub(r"\bQL\.", "Quốc Lộ ", st)
        st = re.sub("-", "/", st)
        st = st.strip()  # Loại bỏ khoảng trắng ở hai đầu của đường dẫn
        lower_text = st.lower()
        if not re.search(r'^chung cư|^hẻm|^ấp|^khu phố|^phố|^quốc lộ|^tổ', lower_text) and not re.search(r'^đường', lower_text):
            st = "Đường " + st
        return st
    return street

In [73]:
result_df_1["cleaned_street"] = result_df_1.apply(lambda x: cleaning_street(x.address_street), axis=1)

In [1]:
import pandas as pd

In [34]:
df = pd.read_excel(r"C:\Users\thinh.lv\google-maps-scraper\src\df_fnb_ninh_kieu.xlsx")

In [35]:
df = df.drop(["Unnamed: 0.1", "Unnamed: 0", "cleaned_street", "address_ward", "address_street", "address_city","address_postal_code", "address_state", "address_country_code","ward", "city"], axis=1)

In [36]:
df.head(5)

,place_id,name,main_category,phone,address
0,ChIJ1zgaajWTpzERAwdFNM7oyxM,Sailing Club Phú Quốc,Nhà hàng,0931 031 035,"Next to Intercontinental, Lô B7 Khu phức, Dươn..."
1,ChIJzcWlVKlioDER8x5DgJZRmzc,Karaoke Paradise 1,Karaoke,0292 3797 797,"18 Lê Lợi, Cái Khế, Ninh Kiều, Cần Thơ"
2,ChIJd3KrYa0odTER_3IUi3FL3s8,Birdy,Quán bar,0938 818 156,"80 Phạm Viết Chánh, Phường 19, Bình Thạnh, Thà..."
3,ChIJcx38zHmJoDERHUu5SybjbXk,QUÁN NGON CẦN THƠ HÒA TUỘC,Nhà hàng,1900 866808,"333 Đ. Nguyễn Văn Linh, Phường An Khánh, Ninh ..."
4,ChIJ4-uslaNioDERJcQkA493bD4,Ninh Kieu Sky Bar,Quán bar,0939 786 123,"02 Hai Ba Trung Stress District City, Tân An, ..."


In [37]:
df

,place_id,name,main_category,phone,address
0,ChIJ1zgaajWTpzERAwdFNM7oyxM,Sailing Club Phú Quốc,Nhà hàng,0931 031 035,"Next to Intercontinental, Lô B7 Khu phức, Dươn..."
1,ChIJzcWlVKlioDER8x5DgJZRmzc,Karaoke Paradise 1,Karaoke,0292 3797 797,"18 Lê Lợi, Cái Khế, Ninh Kiều, Cần Thơ"
2,ChIJd3KrYa0odTER_3IUi3FL3s8,Birdy,Quán bar,0938 818 156,"80 Phạm Viết Chánh, Phường 19, Bình Thạnh, Thà..."
3,ChIJcx38zHmJoDERHUu5SybjbXk,QUÁN NGON CẦN THƠ HÒA TUỘC,Nhà hàng,1900 866808,"333 Đ. Nguyễn Văn Linh, Phường An Khánh, Ninh ..."
4,ChIJ4-uslaNioDERJcQkA493bD4,Ninh Kieu Sky Bar,Quán bar,0939 786 123,"02 Hai Ba Trung Stress District City, Tân An, ..."
...,...,...,...,...,...
11077,ChIJKYx768yJoDERayASQHX00N0,Duy Thái Linh Kiện Điện Tử & Vật Phẩm Trang Trí,Cửa hàng bán đồ điện tử,NaN,2Q68+2C3 Duy Thái Linh Kiện Điện Tử & Vật Phẩm...
11078,ChIJuRtjZiKIoDERxs7Jrnvhfew,Ngọc An Mobile,Cửa hàng điện thoại di động,+84 919 751 975,"Ngọc An Mobile, 21 Đ. 3 Tháng 2, Hưng Lợi, Nin..."
11079,ChIJi9CXQiKIoDERLf0UcIO5JuU,Trung Tâm Viễn Thông 1,Cửa hàng điện thoại di động,+84 292 3833 333,"Trung Tâm Viễn Thông 1, 1, Đường 3/2, Phường X..."
11080,ChIJmaHa3B-IoDERK8998vRvj5E,Cửa Hàng Điện Thoại Di Động Minh Đăng,Cửa hàng bán buôn,+84 292 2223 750,"Cửa Hàng Điện Thoại Di Động Minh Đăng, 86/30, ..."


In [39]:
df

,place_id,name,main_category,phone,address
0,ChIJ1zgaajWTpzERAwdFNM7oyxM,Sailing Club Phú Quốc,Nhà hàng,0931 031 035,"Next to Intercontinental, Lô B7 Khu phức, Dươn..."
1,ChIJzcWlVKlioDER8x5DgJZRmzc,Karaoke Paradise 1,Karaoke,0292 3797 797,"18 Lê Lợi, Cái Khế, Ninh Kiều, Cần Thơ"
2,ChIJd3KrYa0odTER_3IUi3FL3s8,Birdy,Quán bar,0938 818 156,"80 Phạm Viết Chánh, Phường 19, Bình Thạnh, Thà..."
3,ChIJcx38zHmJoDERHUu5SybjbXk,QUÁN NGON CẦN THƠ HÒA TUỘC,Nhà hàng,1900 866808,"333 Đ. Nguyễn Văn Linh, Phường An Khánh, Ninh ..."
4,ChIJ4-uslaNioDERJcQkA493bD4,Ninh Kieu Sky Bar,Quán bar,0939 786 123,"02 Hai Ba Trung Stress District City, Tân An, ..."
...,...,...,...,...,...
11077,ChIJKYx768yJoDERayASQHX00N0,Duy Thái Linh Kiện Điện Tử & Vật Phẩm Trang Trí,Cửa hàng bán đồ điện tử,NaN,"2Q68+2C3 Đ. Trần Hoàng Na, Hưng Lợi, Ninh Kiều..."
11078,ChIJuRtjZiKIoDERxs7Jrnvhfew,Ngọc An Mobile,Cửa hàng điện thoại di động,+84 919 751 975,"21 Đ. 3 Tháng 2, Hưng Lợi, Ninh Kiều, Cần Thơ,..."
11079,ChIJi9CXQiKIoDERLf0UcIO5JuU,Trung Tâm Viễn Thông 1,Cửa hàng điện thoại di động,+84 292 3833 333,"1, Đường 3/2, Phường Xuân Khánh, Quận Ninh Kiề..."
11080,ChIJmaHa3B-IoDERK8998vRvj5E,Cửa Hàng Điện Thoại Di Động Minh Đăng,Cửa hàng bán buôn,+84 292 2223 750,"86/30, Lý Tự Trọng, phường An Cư, quận Ninh Ki..."


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11082 entries, 0 to 11081
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   place_id       11082 non-null  object
 1   name           11082 non-null  object
 2   main_category  10972 non-null  object
 3   phone          8349 non-null   object
 4   address        11041 non-null  object
dtypes: object(5)
memory usage: 433.0+ KB


In [38]:
df['address'] = df.apply(lambda row: str(row['address']).replace(str(row['name']) + ", ", ''), axis=1)

In [41]:
df = df.drop_duplicates()

In [42]:
df.to_excel("df_ninh_kieu.xlsx")